In [23]:
h = {
    'Host': 'cafe.naver.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
    'Sec-Fetch-Dest': 'document',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Referer': 'https://search.naver.com/search.naver?where=article&sm=tab_jum&query=%EC%A0%95%EC%84%B8%EA%B7%A0',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,es;q=0.6'
}

c = {'NNB':'DQFP6L36RAQF4',
     'NRTK':'ag#all_gr#1_ma#-2_si#0_en#0_sp#0',
     'ASID':'7bd04efb0000016fbde0ab4300000053',
     'ncvid':'#vid#_123.208.78.251zb4M',
     'nx_ssl':'2',
     'nsr_acl':'1',
     '_ga_7VKFYR6RV1':'GS1.1.1584658666.23.1.1584658815.59',
     '_ga':'GA1.1.1145665504.1579440103',
     '_naver_usersession_':'oqxxaYeTj2vcYX7B3uHI7w==',
     'page_uid':'UFUNRdprvTVssmcGTLsssssssg8-430560',
     'nci4':'f0cd2a031150398ed53b7c6b668eeb9431629e7e065c9b60acd0dc3d8bdf3af038a714c80fc7585202aee5348a33a5548ead93bb5f4009101fd9718b6d6c99294e694c7d5d45486f46773b384d40785b69285a557453602f606f4a6a58696a6245614e030202040507060e7b7255703e73761a7a7e7e147f7f60630d6265616440; ncvc2=5838c0e4b08687361ce19caf9a6a097fc9877e9886ca7dd9449da370cded0ac70ff432e9330b98b1c55a0dd61fb512cc23582628cfb1a2e5d410ab6ba9903de482958a8c8fc5de89ff9f81888e8d929c98938c918a959291aba8a8aaa8b2',
     'JSESSIONID':'CF74265671F0659CBAC01AAB35D73DE8'}

In [24]:
import requests

response = requests.get("https://cafe.naver.com/poilmom/96011"
                       , headers=h, cookies=c)



In [27]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

a = soup.find(class_="p-nick")


In [28]:
a.get_text()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [30]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="ko">\n <head>\n  <meta content="noindex, nofollow" name="robots"/>\n  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="requiresActiveX=true" http-equiv="X-UA-Compatible"/>\n  <title>\n   Æ÷ÀÏ¸¾½ºÅ¬·´ : ³×ÀÌ¹ö Ä«Æä\n  </title>\n  <link href="/favicon.ico?2" rel="shortcut icon" type="image/x-icon"/>\n  <meta content="always" name="referrer">\n   <link href="/static/css/main/css/line_sticker-1577769455000-9453.css" rel="stylesheet" type="text/css"/>\n   <link href="/static/css/main/css/comment-1577769455000-17199.css" rel="stylesheet" type="text/css"/>\n   <link href="/static/css/main/css/map/cafe_map-1577769455000-14563.css" rel="stylesheet" type="text/css"/>\n   <style id="cafeCss" type="text/css">\n   </style>\n   <link href="/static/css/main/css/wide/1080_cafe-1578547956000-216013.css" rel="stylesheet" type="text/css"/>\n   <link href="/static/css/main/css/wide/1080_skin_pink2-1577769455000-24488.css" rel="stylesheet" 

In [31]:
import requests, time, telegram, re
from bs4 import BeautifulSoup
  
## requests와 BeautifulSoup으로 카페 게시글 목록을 불러오는 함수 ##
def bs(url, page):
    headers = { # 헤더를 넣지 않아도 작동하는 것을 확인했습니다.
        'Content-Type': 'application/json; charset=utf-8',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
    params = {
        'search.clubid' : '18786605', # 카페 ID
        'search.menuid' : '741', # 메뉴 ID
        'search.boardtype' : 'L', # 보드타입 (반드시 필요로 하는 것은 아닌 것 같습니다.)
        'search.page' : page # 불러올 페이지
    }
    html = requests.get(url, headers = headers, params = params)
    soup = BeautifulSoup(html.text, 'html.parser')
    return soup
 
## 게시글 제목과 리플 수를 파싱하여 리플 수가 일정 이상인 경우만 추출하는 함수 ##
def parse(url):
    page = 0
    result_title = []
    result_reply = []
    while page <= 3: # 1~3페이지를 불러옵니다.
        soup = bs(url, page)
        titles = soup.select('strong.tit') # 게시글 제목
        for title in titles:
            result_title.append(' '.join(title.text.strip().split()))
        replys = soup.select('em.num') # 리플 수
        for reply in replys:
            result_reply.append(reply.text)
        page += 1
        time.sleep(0.5)
    count = 0
    final_list = []
    while count < len(result_title):
        if int(result_reply[count]) >= 70: # 리플이 70개가 넘는 글만 리스트에 담습니다.
            final_list.append(result_title[count] + ' (' + result_reply[count] + ')')
        count += 1
    return final_list
 
## 최종결과 문자열에서 리플 수를 제외하는 정규식 처리 ##
def regex(string):
    try: # 괄호 안 숫자가 두 글자인 경우
        if re.compile(r'.+(?<=\(\d{2}\))').search(string).end() > 0:
            result = string[:-5]
    except: # 괄호 안 숫자가 세 글자인 경우 (네 글자인 경우는 상정하지 않았습니다.)
        try:
            if re.compile(r'.+(?<=\(\d{3}\))').search(string).end() > 0:
                result = string[:-6]
        except:
            pass
    return result
 
## TXT 파일로 결과를 저장하고 텔레그램으로 새 글을 알리는 함수 ##
def telegram_bot(titles):
    bot = telegram.Bot(token='토큰을 입력하세요')
    try:
        chat_id = bot.getUpdates()[-1].message.chat.id
    except:
        chat_id = 챗아이디를 입력하세요 # 알림이 장시간 없는 경우에 발생하는 에러를 방지합니다.
 
    try:
        lines = [line.rstrip('\n') for line in open('ncafe.txt', 'r', encoding='utf8')]
    except: # 파일이 존재하지 않는 경우를 예외처리합니다. (처음 실행하는 경우 에러가 발생하기 때문입니다.)
        lines = ['no data']
    
    check = 0
    with open('ncafe.txt', 'w', encoding='utf8') as f: # TXT 파일을 업데이트합니다.
        for title in titles:
            for line in lines:
                if regex(title) == regex(line):
                    check = 1
            if check == 0: # 새 글만 텔레그램으로 알립니다.
                bot.sendMessage(chat_id=chat_id, text=title)
                print(title)
            f.write(title  + '\n')
 
if __name__ == '__main__':
    url = 'https://m.cafe.naver.com/ArticleAllListAjax.nhn'
    titles = parse(url)
    telegram_bot(titles)

SyntaxError: invalid syntax (<ipython-input-31-7d34610d7cf4>, line 63)

In [32]:
#https://wikidocs.net/67133